# Let's make NMT

All for One

Choi Minjoo (Judy Choi)

## Mount Drive & Files

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
my_path = '/content/notebooks'
os.symlink('/content/drive/MyDrive/AllforOne/package_collection', my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


## 1. Dataset
AI Hub 'Korean-English pair corpus'

https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=126

### Split Dataset
*   Val(Dev) : 5,000
*   Test : 3,000
*   Train : 1,493,750

Simple view of dataset

In [2]:
cd /content/drive/MyDrive/AllforOne/Lecture/Day4

/content/drive/MyDrive/AllforOne/Lecture/Day4


In [3]:
!wc -l dataset/train.en

1493750 dataset/train.en


In [4]:
!head dataset/train.en

"No problem, but may I ask you why if you don't mind?"
"Even if I'm careful, it's hard to predict overspeeding vehicles."
"Entering the expected expenses for reading books and watching performances will confirm the estimated amount of income tax deduction refunds, and the participating members will be given a 1,000 won voucher that can be used on Yes24."
"Lee Ki-woo, deputy governor for social integration, will head the committee, and its members are composed of 14 members, including Park Ok-bun of the provincial council, Han Ok-ja, director of the Gyeonggi Provincial Family Research Institute, Park Jeong-ran, director of the Women's Family Bureau, and outside experts."
"The supporters cheer at the Democratic Party's candidate office of Yang Seung-jo, governor of Cheonan, South Chungcheong Province, at 6 p.m. on the 13rd, when the results of the exit polls for the June 13 local elections were announced."
"When I call the Education Support Service, I often hear, ""There are more multicu

## 2. Subword Tokenization

### SentencePiece
* Unsupervised text tokenizer and detokenize
* Not depend on language
* Not depend on Spacing or not
* Alleviate the open vocabulary problems (OOV)
* Supports **BPE(Byte-Pair-Encoding), Unigram** language model

#### Install
Build and install SentencePiece command line tools from C++ source

In [5]:
!git clone https://github.com/google/sentencepiece.git 

fatal: destination path 'sentencepiece' already exists and is not an empty directory.


In [6]:
cd sentencepiece

/content/drive/MyDrive/AllforOne/Lecture/Day4/sentencepiece


In [7]:
mkdir build

mkdir: cannot create directory ‘build’: File exists


In [8]:
cd build

/content/drive/MyDrive/AllforOne/Lecture/Day4/sentencepiece/build


In [9]:
!cmake ..

CMake Error: The current CMakeCache.txt directory /content/drive/MyDrive/AllforOne/Lecture/Day4/sentencepiece/build/CMakeCache.txt is different than the directory /content/drive/MyDrive/AllforOne/NMT_ko-en/sentencepiece/build where CMakeCache.txt was created. This may result in binaries being created in the wrong place. If you are not sure, reedit the CMakeCache.txt
CMake Error: The source "/content/drive/MyDrive/AllforOne/Lecture/Day4/sentencepiece/CMakeLists.txt" does not match the source "/content/drive/MyDrive/AllforOne/NMT_ko-en/sentencepiece/CMakeLists.txt" used to generate cache.  Re-run cmake with a different source directory.


In [10]:
!make -j $(nproc)

/bin/sh: 1: cd: can't cd to /content/drive/MyDrive/AllforOne/NMT_ko-en/sentencepiece/build
src/CMakeFiles/sentencepiece.dir/build.make:739: recipe for target 'src/CMakeFiles/sentencepiece.dir/depend' failed
make[2]: *** [src/CMakeFiles/sentencepiece.dir/depend] Error 2
CMakeFiles/Makefile2:131: recipe for target 'src/CMakeFiles/sentencepiece.dir/all' failed
make[1]: *** [src/CMakeFiles/sentencepiece.dir/all] Error 2
make[1]: *** Waiting for unfinished jobs....
/bin/sh: 1: cd: can't cd to /content/drive/MyDrive/AllforOne/NMT_ko-en/sentencepiece/build
src/CMakeFiles/sentencepiece_train-static.dir/build.make:253: recipe for target 'src/CMakeFiles/sentencepiece_train-static.dir/depend' failed
make[2]: *** [src/CMakeFiles/sentencepiece_train-static.dir/depend] Error 2
CMakeFiles/Makefile2:209: recipe for target 'src/CMakeFiles/sentencepiece_train-static.dir/all' failed
make[1]: *** [src/CMakeFiles/sentencepiece_train-static.dir/all] Error 2
/bin/sh: 1: cd: can't cd to /content/drive/MyDrive

In [11]:
!sudo make install

/bin/sh: 1: cd: can't cd to /content/drive/MyDrive/AllforOne/NMT_ko-en/sentencepiece/build
src/CMakeFiles/sentencepiece.dir/build.make:739: recipe for target 'src/CMakeFiles/sentencepiece.dir/depend' failed
make[2]: *** [src/CMakeFiles/sentencepiece.dir/depend] Error 2
CMakeFiles/Makefile2:131: recipe for target 'src/CMakeFiles/sentencepiece.dir/all' failed
make[1]: *** [src/CMakeFiles/sentencepiece.dir/all] Error 2
Makefile:155: recipe for target 'all' failed
make: *** [all] Error 2


In [12]:
!sudo ldconfig -v

/sbin/ldconfig.real: Can't stat /usr/local/cuda-11.1/extras/CUPTI/lib64: No such file or directory
/sbin/ldconfig.real: Path `/usr/local/lib' given more than once
/sbin/ldconfig.real: Can't stat /usr/local/nvidia/lib: No such file or directory
/sbin/ldconfig.real: Can't stat /usr/local/nvidia/lib64: No such file or directory
/sbin/ldconfig.real: Can't stat /usr/local/lib/x86_64-linux-gnu: No such file or directory
/sbin/ldconfig.real: Path `/lib/x86_64-linux-gnu' given more than once
/sbin/ldconfig.real: Path `/usr/lib/x86_64-linux-gnu' given more than once
/usr/local/cuda-11.1/targets/x86_64-linux/lib:
	libnvrtc-builtins.so.11.1 -> libnvrtc-builtins.so.11.1.105
	libcusolver.so.11 -> libcusolver.so.11.0.1.105
	libcufftw.so.10 -> libcufftw.so.10.3.0.105
	libcusparse.so.11 -> libcusparse.so.11.3.0.10
	libnvToolsExt.so.1 -> libnvToolsExt.so.1.0.0
	libnppidei.so.11 -> libnppidei.so.11.1.2.301
	libcublasLt.so.11 -> libcublasLt.so.11.3.0.106
	libnppisu.so.11 -> libnppisu.so.11.1.2.301
	libnp

#### Train
Trains model from raw sentences

Parameter
*   input : data file name
*   model_prefix : spm model name
*   vocab size 32000

In [ ]:
!spm_train --input=train.ko --model_prefix=korean --vocab_size=32000 --character_coverage=1.0 --train_extremely_large_corpus --input_sentence_size=1000000 and --shuffle_input_sentence=true
!spm_train --input=train.en --model_prefix=english --vocab_size=32000 --character_coverage=1.0 --train_extremely_large_corpus --input_sentence_size=1000000 and --shuffle_input_sentence=true

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: train.ko
  input_format: 
  model_prefix: korean
  model_type: UNIGRAM
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 1000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 1
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy_noise_level: 0
  differential_privacy_clipping_

#### Encoding
Encode raw text into sentence pieces/ids
* Subword Segmentation
* Whitespace " " is converted to a meta symbol "▁" (U+2581)

ex)
* (Before) The comfortable machine with single-wheel suspension.
* (After) ▁The ▁comfortable ▁machine ▁with ▁single - wheel ▁suspension .

In [ ]:
!spm_encode --model korean.model --output train.ko.encoded train.ko
!spm_encode --model korean.model --output val.ko.encoded val.ko
!spm_encode --model korean.model --output test.ko.encoded test.ko

In [ ]:
!spm_encode --model english.model --output train.en.encoded train.en
!spm_encode --model english.model --output val.en.encoded val.en
!spm_encode --model english.model --output test.en.encoded test.en

## 3. Model

### OpenNMT-py
Open-source (MIT) neural machine translation framework

https://opennmt.net/


* Highly configurable model architectures and training procedures
* Efficient model serving capabilities for use in real world applications
* Extensions to allow other tasks such as text generation, tagging, summarization, image to text, and speech to text
* Pytorch / TensorFlow

####Install pytorch

In [13]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


####Install OpenNMT-py

In [14]:
!pip install OpenNMT-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 13.8 MB/s 
     |████████████████████████████████| 16.6 MB 37.1 MB/s 
     |████████████████████████████████| 57 kB 7.0 MB/s 
     |████████████████████████████████| 73 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 82.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.0
    Uninstalling torchtext-0.13.0:
      Successfully uninstalled torchtext-0.13.0


#### Configuration file
Build a YAML configuration file (config.yml) to specify the data that will be used

#### Build vocab
Build the vocab(s) that will be necessary to train the model.

Build the vocab(s) following to the YAML configuration file (config.yml).

In [ ]:
!onmt_build_vocab -config config.yml -n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-07-15 07:12:21,507 INFO] Counter vocab from -1 samples.
[2022-07-15 07:12:21,507 INFO] n_sample=-1: Build vocab on full datasets.
[2022-07-15 07:12:21,516 INFO] corpus_1's transforms: TransformPipe()
[2022-07-15 07:13:07,921 INFO] Counters src:32348
[2022-07-15 07:13:07,921 INFO] Counters tgt:32071


#### Train
Train NMT model following to the YAML configuration file (config.yml)

In [ ]:
!onmt_train -config config.yml

[2022-07-15 07:13:10,325 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-07-15 07:13:10,326 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-07-15 07:13:10,326 INFO] Missing transforms field for valid data, set to default: [].
[2022-07-15 07:13:10,326 INFO] Parsed 2 corpora from -data.
[2022-07-15 07:13:10,327 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-07-15 07:13:10,327 INFO] Loading vocab from text file...
[2022-07-15 07:13:10,327 INFO] Loading src vocabulary from vocab/vocab.ko
[2022-07-15 07:13:10,401 INFO] Loaded src vocab has 32348 tokens.
[2022-07-15 07:13:10,417 INFO] Loading tgt vocabulary from vocab/vocab.en
[2022-07-15 07:13:10,516 INFO] Loaded tgt vocab has 32071 tokens.
[2022-07-15 07:13:10,534 INFO] Building fields with vocab in counters...
[2022-07-15 07:13:10,593 INFO]  * tgt vocab size: 32075.
[2022-07-15 07:13:10,643 INFO]  * src vocab size: 32350.
[2022-07-15 07:13:10,6

#### Traslate (Generate)
Now we have a model which you can use to predict on new data.

We do this by running beam search.

In [ ]:
!onmt_translate -model ko-en_step_80000.pt -src test.ko.encoded -output pred.ko-en --replace_unk --verbose --max_length=512

[2022-07-16 04:05:44,240 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size



Test file is too big (3,000 sentences) .

So I'll translate just 100 sentences.

In [ ]:
!head -n 100 test.ko.encoded > test_100.ko.encoded
!head -n 100 test.en > test_100.en

In [ ]:
!onmt_translate -model ko-en_step_80000.pt -src test_100.ko.encoded -output pred_100.ko-en --replace_unk --verbose --max_length=512

[2022-07-16 04:06:29,832 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-07-16 04:06:35,417 INFO] 
SENT 1: ['▁제가', '▁보', '니깐', '▁다', '래', '끼', '처럼', '▁보이네요', '.', '▁지금은', '▁초기', '라서', '▁덜', '▁부어', '있는', '▁상태', '입니다', '.']
PRED 1: ▁" It ▁looks ▁like ▁a ▁sty ▁when ▁I ▁look ▁at ▁it , ▁but ▁it ' s ▁early ▁so ▁it ' s ▁less ▁swollen ."
PRED SCORE: -11.8514

[2022-07-16 04:06:35,417 INFO] 
SENT 2: ['▁"', '이', '▁밖에', '도', '▁약', '수', '시장', '▁곳곳에서', '▁대형', '윷', '놀이', ',', '▁제기', '차', '기', ',', '▁투', '호', '놀이', '▁등', '▁전

## 4. Detokenization
Decode sentence pieces/ids into raw text

In [ ]:
!spm_decode --model english.model --output pred.ko-en.decoded pred.ko-en

spm_decode_main.cc(54) [_status.ok()] Not found: "french.model": No such file or directory Error #2
Program terminated with an unrecoverable error.


In [ ]:
!spm_decode --model english.model --output pred_100.ko-en.decoded pred_100.ko-en

In [ ]:
!head pred_100.ko-en.decoded

"It looks like a sty when I look at it, but it's early so it's less swollen."
"In addition, traditional games such as large yut nori, Jegichagi and tuho nori will be held throughout the Yaksu market."
The investigation referred to in paragraph (1) shall be made in writing jointly by the proposing member under the investigation plan under Article 41 of the Decree.
"The Korea Architecture and Culture Awards, which have been organized annually by the Ministry of Land, Infrastructure and Transport, the Korea Institute of Architects and the Seoul Economic Daily, have already been established as a window for the current and future of architecture in this country."
I almost sleep on Sunday morning.
"President Emmanuel Macron, who claims to play the role of a ""French bottle,"" is drawing attention as a speech practice video criticizing the social security budget that simply supports the poor."
"Mogenso's claim may have been valid before and after the liberation, when state capabilities were w

## 5. Evaluation

### BLEU
Quantitative evaluation is performed on the sentence thus obtained. BLEU is a quantitative evaluation method for machine translation. You can see which model is superior by comparing it to the BLEU score you are comparing.

https://www.aclweb.org/anthology/P02-1040

In [ ]:
!perl  multi-bleu.perl test.fr < pred.en-fr.decoded

/bin/bash: pred.en-fr.decoded: No such file or directory


In [ ]:
!perl  multi-bleu.perl test_100.en < pred_100.ko-en.decoded

BLEU = 35.97, 63.2/42.4/30.5/22.2 (BP=0.980, ratio=0.980, hyp_len=2329, ref_len=2376)
